In [3]:
import torch
from tools import *
import numpy as np
from sklearn.metrics import mutual_info_score
from sklearn.feature_selection import mutual_info_regression

In [2]:
cm = ConfigManager('config.yaml')
trainer = Trainer(cm)

In [3]:
train, test = trainer.load_dataset()

In [4]:
def sample_freq(x, f1, f2):
    b, c, h, w = x.shape
    freq = torch.fft.rfft2(x, dim=(-2, -1), norm='forward')
    x1 = torch.cat([torch.abs(freq[..., f1//2, :f1//2].view(b*c, -1)), torch.abs(freq[..., :f1//2, f1//2].view(b*c, -1)), torch.abs(freq[..., -f1//2, -f1//2:].view(b*c, -1)), torch.abs(freq[..., -f1//2:, -f1//2].view(b*c, -1))], dim=-1)
    x2 = torch.cat([torch.abs(freq[..., f2//2, :f2//2].view(b*c, -1)), torch.abs(freq[..., :f2//2, f2//2].view(b*c, -1)), torch.abs(freq[..., -f2//2, -f2//2:].view(b*c, -1)), torch.abs(freq[..., -f2//2:, -f2//2].view(b*c, -1))], dim=-1)
    return x1, x2

In [8]:
mi = 0
for i in train:
    x = i[0]
    x1, x2 = sample_freq(x, 28, 27)
    val = 0
    for t in range(x2.shape[-1]):
        X = x1.numpy()
        y = x2[..., t].numpy()
        mi_scores = mutual_info_regression(X, y)
        val += np.mean(mi_scores)
    mi += val / x.shape[-1]
    print(np.mean(mi_scores))
mi /= len(train)


0.1125232385219028
0.11639244318029089
0.11570499253232724
0.11323224943835242
0.11647434633920802
0.11610526670236222
0.11436637279230048
0.11917888925694634
0.1120874111566561
0.11799332807937765
0.11381712719739667


KeyboardInterrupt: 

In [1]:
height=28
width=15
batch_size = 1

In [4]:
x_coords = torch.arange(width, dtype=torch.float32).view(1, 1, 1, -1)
y_coords = torch.arange(height, dtype=torch.float32).view(1, 1, -1, 1)
pos = torch.zeros((batch_size, 2, height, width), dtype=torch.float32)
pos[:, 0, :, :] = x_coords / 2
pos[:, 1, :, :] = y_coords / 2

In [5]:
midpoint = height // 2

# Flip the upper half vertically and copy to the lower half
pos[:, 1, midpoint:, :] = torch.flip(pos[:, 1, :midpoint, :], [-2])

In [6]:
torch.exp(torch.maximum(pos[:, 0, ...], pos[:, 1, ...])/torch.max(pos) - 1)

tensor([[[0.3679, 0.3951, 0.4244, 0.4558, 0.4895, 0.5258, 0.5647, 0.6065,
          0.6514, 0.6997, 0.7515, 0.8071, 0.8669, 0.9311, 1.0000],
         [0.3951, 0.3951, 0.4244, 0.4558, 0.4895, 0.5258, 0.5647, 0.6065,
          0.6514, 0.6997, 0.7515, 0.8071, 0.8669, 0.9311, 1.0000],
         [0.4244, 0.4244, 0.4244, 0.4558, 0.4895, 0.5258, 0.5647, 0.6065,
          0.6514, 0.6997, 0.7515, 0.8071, 0.8669, 0.9311, 1.0000],
         [0.4558, 0.4558, 0.4558, 0.4558, 0.4895, 0.5258, 0.5647, 0.6065,
          0.6514, 0.6997, 0.7515, 0.8071, 0.8669, 0.9311, 1.0000],
         [0.4895, 0.4895, 0.4895, 0.4895, 0.4895, 0.5258, 0.5647, 0.6065,
          0.6514, 0.6997, 0.7515, 0.8071, 0.8669, 0.9311, 1.0000],
         [0.5258, 0.5258, 0.5258, 0.5258, 0.5258, 0.5258, 0.5647, 0.6065,
          0.6514, 0.6997, 0.7515, 0.8071, 0.8669, 0.9311, 1.0000],
         [0.5647, 0.5647, 0.5647, 0.5647, 0.5647, 0.5647, 0.5647, 0.6065,
          0.6514, 0.6997, 0.7515, 0.8071, 0.8669, 0.9311, 1.0000],
         [0.6